<a href="https://colab.research.google.com/github/rar8393/NLG_tests/blob/main/roberta_model_combo_finetune_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#https://github.com/huggingface/notebooks/blob/main/examples/language_modeling.ipynb

In [2]:
!pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import transformers

print(transformers.__version__)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

4.23.1


In [4]:
from datasets import load_dataset
datasets = load_dataset("text", data_files={"train": 'train_health_legal.txt', "validation": 'val_health_legal.txt'})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-6a79ef11f98abae4/0.0.0/21a506d1b2b34316b1e82d0bd79066905d846e5d7e619823c0dd338d6f1fa6ad. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [6]:
show_random_elements(datasets["train"])

In [7]:
#"facebook/opt-125m". "saibo/legal-roberta-base"
model_checkpoint = "roberta-base"

In [8]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [9]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [10]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

#0:   0%|          | 0/9 [00:00<?, ?ba/s]

#1:   0%|          | 0/9 [00:00<?, ?ba/s]

#2:   0%|          | 0/9 [00:00<?, ?ba/s]

#3:   0%|          | 0/9 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (530 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (660 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (555 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (542 > 512). Running this sequence through the model will result in indexing errors


#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (561 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (575 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (631 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (564 > 512). Running this sequence through the model will result in indexing errors


In [11]:
# block_size = tokenizer.model_max_length
block_size = 512

In [12]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [13]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

#1:   0%|          | 0/9 [00:00<?, ?ba/s]

#0:   0%|          | 0/9 [00:00<?, ?ba/s]

#2:   0%|          | 0/9 [00:00<?, ?ba/s]

#3:   0%|          | 0/9 [00:00<?, ?ba/s]

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

In [14]:
from transformers import AutoModelForCausalLM
#If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`
model = AutoModelForCausalLM.from_pretrained(model_checkpoint, is_decoder=True)

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

In [15]:
from transformers import Trainer, TrainingArguments

In [16]:
#https://stackoverflow.com/questions/72680734/huggingface-trainer-only-doing-3-epochs-no-matter-the-trainingarguments
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-combo-1",
    evaluation_strategy = "steps", #epoch
    num_train_epochs=6, 
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=False,
    warmup_steps=1000,                # number of warmup steps for learning rate  
    save_steps=2000,
    eval_steps=500,
    save_total_limit=10,
    load_best_model_at_end= True,
    #output_dir='./results',
)

In [17]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

In [18]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 21881
  Num Epochs = 6
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 16416


Step,Training Loss,Validation Loss
500,5.644500,3.931845
1000,3.892600,3.486909
1500,3.584700,3.289219
2000,3.417700,3.183686
2500,3.332200,3.107797
3000,3.240100,3.057080
3500,3.171700,3.014342
4000,3.138300,2.983114
4500,3.096600,2.948438
5000,3.070100,2.923861


***** Running Evaluation *****
  Num examples = 3503
  Batch size = 8
***** Running Evaluation *****
  Num examples = 3503
  Batch size = 8
***** Running Evaluation *****
  Num examples = 3503
  Batch size = 8
***** Running Evaluation *****
  Num examples = 3503
  Batch size = 8
Saving model checkpoint to roberta-base-combo-1/checkpoint-2000
Configuration saved in roberta-base-combo-1/checkpoint-2000/config.json
Model weights saved in roberta-base-combo-1/checkpoint-2000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 3503
  Batch size = 8
***** Running Evaluation *****
  Num examples = 3503
  Batch size = 8
***** Running Evaluation *****
  Num examples = 3503
  Batch size = 8
***** Running Evaluation *****
  Num examples = 3503
  Batch size = 8
Saving model checkpoint to roberta-base-combo-1/checkpoint-4000
Configuration saved in roberta-base-combo-1/checkpoint-4000/config.json
Model weights saved in roberta-base-combo-1/checkpoint-4000/pytorch_model.bin
***** Runnin

KeyboardInterrupt: ignored

In [19]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("roberta-base-combo-1/checkpoint-8000")

loading configuration file roberta-base-combo-1/checkpoint-8000/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base-combo-1/checkpoint-8000",
  "architectures": [
    "RobertaForCausalLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": true,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.23.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file roberta-base-combo-1/checkpoint-8000/pytorch_model.bin
All model checkpoint weights were used when initializing RobertaForCausalLM.

All the weights of Rober

In [22]:
import importlib
import generate_txt
importlib.reload(generate_txt)
generator = generate_txt.generate_model_results(tokenizer, model)

In [23]:
test_metrics_1 = generator.generate_sample("The event has a ", visualize=False)

/content/generated_text_metrics.py:52: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity.append(gen_doc.similarity(sent))
/content/generated_text_metrics.py:54: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Span.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one 

In [24]:
for key in test_metrics_1.keys():
  print(key)

The event has a 
the first ground of appeal, the applicant also relies upon the following observation regarding some other authorities: (h) the first ground is of doubtful prospect.
a)                    [***].  We  have                                   
The question of whether  it is the legal intention of the Tribunal to make an order should be determined: Applicant S157/2002 v Minister for Immigration and Ethnic Affairs (2004) 210 ALR 3 at [39] ; SZBYR v Minister for Immigration and Citizenship [2004] FCAFC 54 ;
7.2.1 The indemnity or other benefit thereof (the "Income") may not be awarded or claimed if it is an indemnified party as a result of the Company, in respect of an amount of $[***] or such other amount as is not incurred, directly or indirectly, and (
The effect of this section, having regard to the provision of the provisions of s 31A(1), does not apply to the conduct of the exercise of one's power. The Act provides that this provision is limited because it cannot be inte

In [25]:
for key in test_metrics_1.keys():
  print(test_metrics_1[key]['metric'])

{'number_ents': 0, 'ents': [], 'number_nouns': 1, 'nouns': ['event'], 'number_sents': 1, 'avg_sent_len': 4.0, 'sent_repeat_score': [0.0]}
{'number_ents': 2, 'ents': [first, first], 'number_nouns': 8, 'nouns': ['ground', 'appeal', 'applicant', 'observation', 'authority', 'h', 'ground', 'prospect'], 'number_sents': 1, 'avg_sent_len': 30.0, 'sent_repeat_score': [0.19999999999999996]}
{'number_ents': 0, 'ents': [], 'number_nouns': 0, 'nouns': [], 'number_sents': 3, 'avg_sent_len': 4.666666666666667, 'sent_repeat_score': [0.0, 0.2857142857142857, 0.0]}
{'number_ents': 3, 'ents': [Tribunal, 2004, 39], 'number_nouns': 17, 'nouns': ['question', 'intention', 'Tribunal', 'order', 'Applicant', 's157/2002', 'Minister', 'Immigration', 'Ethnic', 'Affairs', 'alr', 'SZBYR', 'v', 'Minister', 'Immigration', 'Citizenship', 'FCAFC'], 'number_sents': 2, 'avg_sent_len': 27.0, 'sent_repeat_score': [0.1875, 0.0]}
{'number_ents': 0, 'ents': [], 'number_nouns': 10, 'nouns': ['indemnity', 'benefit', 'income', 'p

In [26]:
print('Number named entities, number nouns, number of sentences, avg sentence length, repeat score')
for key in test_metrics_1.keys():
  print(test_metrics_1[key]['metric']['number_ents'], test_metrics_1[key]['metric']['number_nouns'],
        test_metrics_1[key]['metric']['number_sents'],  test_metrics_1[key]['metric']['avg_sent_len'],
        test_metrics_1[key]['metric']['sent_repeat_score'])

Number named entities, number nouns, number of sentences, avg sentence length, repeat score
0 1 1 4.0 [0.0]
2 8 1 30.0 [0.19999999999999996]
0 0 3 4.666666666666667 [0.0, 0.2857142857142857, 0.0]
3 17 2 27.0 [0.1875, 0.0]
0 10 2 29.5 [0.0, 0.3448275862068966]
2 14 3 20.333333333333332 [0.33333333333333337, 0.11111111111111116, 0.0]


In [27]:
test_metrics_2 = generator.generate_sample("The conference started with a ", visualize=False)

In [28]:
for key in test_metrics_2.keys():
  print(key)

The conference started with a 
13.10 "Trademark Rights" of the Company, as defined below. The Company hereby grants to the Agent, no later than ninety (90) days prior to the date of the event CTS or CTS determines the use of any Third-Party's Confidential Information.  CTS and C
2.2  For any failure or delay in completion,                                                   
4.6.1                   "CAS) During the  Term, CUSAG may, at its own and                      
15.2                                                            
A final stage of the trial as of December 31, 2015 is the trial as outlined in Section 7.3-4. 4.2.


In [29]:
for key in test_metrics_2.keys():
  print(test_metrics_2[key]['metric'])

{'number_ents': 0, 'ents': [], 'number_nouns': 1, 'nouns': ['conference'], 'number_sents': 1, 'avg_sent_len': 5.0, 'sent_repeat_score': [0.0]}
{'number_ents': 8, 'ents': [13.10, Trademark Rights, Company, 90) days, CTS, CTS, Confidential Information, CTS], 'number_nouns': 17, 'nouns': ['Trademark', 'Rights', 'Company', 'Company', 'Agent', 'day', 'date', 'event', 'CTS', 'CTS', 'use', 'Third', 'Party', 'Confidential', 'Information', 'CTS', 'c'], 'number_sents': 3, 'avg_sent_len': 18.333333333333332, 'sent_repeat_score': [0.07692307692307687, 0.17948717948717952, 0.0]}
{'number_ents': 1, 'ents': [2.2  ], 'number_nouns': 3, 'nouns': ['failure', 'delay', 'completion'], 'number_sents': 1, 'avg_sent_len': 11.0, 'sent_repeat_score': [0.0]}
{'number_ents': 1, 'ents': [CUSAG], 'number_nouns': 3, 'nouns': ['CAS', 'Term', 'CUSAG'], 'number_sents': 1, 'avg_sent_len': 18.0, 'sent_repeat_score': [0.05555555555555558]}
{'number_ents': 1, 'ents': [15.2], 'number_nouns': 0, 'nouns': [], 'number_sents': 

In [30]:
print('Number named entities, number nouns, number of sentences, avg sentence length, repeat score')
for key in test_metrics_2.keys():
  print(test_metrics_2[key]['metric']['number_ents'], test_metrics_2[key]['metric']['number_nouns'],
        test_metrics_2[key]['metric']['number_sents'],  test_metrics_2[key]['metric']['avg_sent_len'],
        test_metrics_2[key]['metric']['sent_repeat_score'])

Number named entities, number nouns, number of sentences, avg sentence length, repeat score
0 1 1 5.0 [0.0]
8 17 3 18.333333333333332 [0.07692307692307687, 0.17948717948717952, 0.0]
1 3 1 11.0 [0.0]
1 3 1 18.0 [0.05555555555555558]
1 0 1 2.0 [0.0]
3 5 1 25.0 [0.19999999999999996]


In [31]:
#event_txt = 'Calling All Climate Warriors! Youth for Climate Justice is hosting a series of webinars where we feature three climate justice-focused initiatives spearheaded by young community organizers. Discover local movements in your community, learn about effective ways to communicate with legislators and lead the charge in climate justice! '
sick_txt = 'The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will '


In [32]:
test_metrics_3 = generator.generate_sample(sick_txt, visualize=False)

In [33]:
for key in test_metrics_3.keys():
  print(key)

The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will 
We report this observational study on the number of patients as well as their severity, during the outbreak and for COVID-19 in China to determine the prognosis of the infection. We evaluated risk factors for the infections, including the disease progression, and the outcome of the trial. The median risk of severe infection was
Introduction: The pandemic of coronavirus disease 2019 (COVID-19) caused by Coronavirus Disease 2019 (COVID-19). To evaluate the effect of coronaviruses against respiratory viruses (GFRs) of a population of COVID-19 patients, we assessed the impact of

Background: The novel coronavirus (COVID-19) pandemic has taken to a global arena in nearly two years, resulting in widespread public health failures. The basic reproduction number is a critical factor in understanding our host’s pathogenesis and understanding virus selection. It is essential to under

In [34]:
for key in test_metrics_3.keys():
  print(test_metrics_3[key]['metric'])

{'number_ents': 1, 'ents': [this year], 'number_nouns': 4, 'nouns': ['flu', 'year', 'conference', 'vaccine'], 'number_sents': 3, 'avg_sent_len': 7.0, 'sent_repeat_score': [0.0, 0.0, 0.0]}
{'number_ents': 1, 'ents': [China], 'number_nouns': 18, 'nouns': ['study', 'number', 'patient', 'severity', 'outbreak', 'covid-19', 'China', 'prognosis', 'infection', 'risk', 'factor', 'infection', 'disease', 'progression', 'outcome', 'trial', 'risk', 'infection'], 'number_sents': 3, 'avg_sent_len': 19.666666666666668, 'sent_repeat_score': [0.15625, 0.19999999999999996, 0.0]}
{'number_ents': 3, 'ents': [2019, Coronavirus Disease, 2019], 'number_nouns': 16, 'nouns': ['introduction', 'pandemic', 'coronavirus', 'disease', 'COVID-19', 'Coronavirus', 'Disease', 'COVID-19', 'effect', 'coronaviruse', 'virus', 'gfr', 'population', 'covid-19', 'patient', 'impact'], 'number_sents': 2, 'avg_sent_len': 22.0, 'sent_repeat_score': [0.19999999999999996, 0.16666666666666663]}
{'number_ents': 0, 'ents': [], 'number_no

In [35]:
print('Number named entities, number nouns, number of sentences, avg sentence length, repeat score')
for key in test_metrics_3.keys():
  print(test_metrics_3[key]['metric']['number_ents'], test_metrics_3[key]['metric']['number_nouns'],
        test_metrics_3[key]['metric']['number_sents'],  test_metrics_3[key]['metric']['avg_sent_len'],
        test_metrics_3[key]['metric']['sent_repeat_score'])

Number named entities, number nouns, number of sentences, avg sentence length, repeat score
1 4 3 7.0 [0.0, 0.0, 0.0]
1 18 3 19.666666666666668 [0.15625, 0.19999999999999996, 0.0]
3 16 2 22.0 [0.19999999999999996, 0.16666666666666663]
0 0 0 0.0 []
1 17 3 18.0 [0.03703703703703709, 0.052631578947368474, 0.0]
2 18 1 46.0 [0.4347826086956522]


In [36]:
event_txt = 'Calling All Climate Warriors! Youth for Climate Justice is hosting a series of webinars where we feature three climate justice-focused initiatives spearheaded by young community organizers. Discover local movements in your community, learn about effective ways to communicate with legislators and lead the charge in climate justice! '
sick_txt = 'The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will '
#generator2(event_txt + sick_txt)
combo = event_txt + sick_txt

In [37]:
test_metrics_4 = generator.generate_sample(combo, visualize=False)

In [38]:
for key in test_metrics_4.keys():
  print(key)

Calling All Climate Warriors! Youth for Climate Justice is hosting a series of webinars where we feature three climate justice-focused initiatives spearheaded by young community organizers. Discover local movements in your community, learn about effective ways to communicate with legislators and lead the charge in climate justice! The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will 
Objective The identification of a novel SARS-CoV-2 infection was performed in two cases. This study investigated the spread of serant RNA RNA into the oral and throat of a people with COVID-19 patients with mild-to-severe COVID-19. Methods To evaluate the prevalence and disease
Background The COVID-19 pandemic has been overwhelming in the world by the late 2017 season. First, as a global epidemic, we experienced a surge of cases from the African continent, which affected the global population. We aimed to use the data of the African African conti

In [39]:
for key in test_metrics_4.keys():
  print(test_metrics_4[key]['metric'])

{'number_ents': 2, 'ents': [three, this year], 'number_nouns': 23, 'nouns': ['Climate', 'Warriors', 'youth', 'Climate', 'Justice', 'series', 'webinar', 'climate', 'justice', 'initiative', 'community', 'organizer', 'movement', 'community', 'way', 'legislator', 'charge', 'climate', 'justice', 'flu', 'year', 'conference', 'vaccine'], 'number_sents': 6, 'avg_sent_len': 12.333333333333334, 'sent_repeat_score': [0.0, 0.0, 0.04347826086956519, 0.0, 0.0, 0.0]}
{'number_ents': 1, 'ents': [two], 'number_nouns': 17, 'nouns': ['identification', 'SARS', 'CoV-2', 'infection', 'case', 'study', 'spread', 'RNA', 'RNA', 'throat', 'people', 'covid-19', 'patient', 'covid-19', 'method', 'prevalence', 'disease'], 'number_sents': 3, 'avg_sent_len': 17.0, 'sent_repeat_score': [0.0, 0.2142857142857143, 0.0]}
{'number_ents': 4, 'ents': [the late 2017 season, First, African, African African], 'number_nouns': 13, 'nouns': ['background', 'covid-19', 'pandemic', 'world', 'season', 'epidemic', 'surge', 'case', 'cont

In [40]:
print('Number named entities, number nouns, number of sentences, avg sentence length, repeat score')
for key in test_metrics_4.keys():
  metric_load = test_metrics_4[key]['metric']
  print(metric_load['number_ents'], metric_load['number_nouns'],
        metric_load['number_sents'],  metric_load['avg_sent_len'],
        metric_load['sent_repeat_score'])

Number named entities, number nouns, number of sentences, avg sentence length, repeat score
2 23 6 12.333333333333334 [0.0, 0.0, 0.04347826086956519, 0.0, 0.0, 0.0]
1 17 3 17.0 [0.0, 0.2142857142857143, 0.0]
4 13 3 19.0 [0.125, 0.20833333333333337, 0.11764705882352944]
0 19 2 29.0 [0.0, 0.2321428571428571]
1 0 1 1.0 [0.0]
1 17 5 11.6 [0.0, 0.050000000000000044, 0.0, 0.0, 0.0]


In [41]:
#check different lengths
length_metric_1 = generator.generate_sample(sick_txt, max_length=32, visualize=False)
length_metric_2 = generator.generate_sample(sick_txt, max_length=64, visualize=False)
length_metric_3 = generator.generate_sample(sick_txt, max_length=128, visualize=False)

In [42]:
for key in length_metric_1.keys():
  print(key)
  print(len(key))

The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will 
114
The authors believe that these findings have the potential to inform the development of a therapeutic vaccine that the immune system efficiently fights.
152
There is no definitive conclusion about this topic regarding the effect of certain medications and other treatments. It is therefore important to note that both legal and medical authorities do
193
Introduction: The study was aimed specifically at exploring the efficacy and safety of the antifungal and antiviral drugs as a treatment in the COVID-
150
The COVID-19 pandemic is an epicenter of uncertainty. The public health is critical, but the global global health problems are exacerbated. The most
148
(a) Notwithstanding anything contained in this Agreement, at the time of delivery this Agreement is delivered to Zogenix, Zogenix has obtained, licensed
152


In [43]:
for key in length_metric_1.keys():
  print(length_metric_1[key]['metric'])

{'number_ents': 1, 'ents': [this year], 'number_nouns': 4, 'nouns': ['flu', 'year', 'conference', 'vaccine'], 'number_sents': 3, 'avg_sent_len': 7.0, 'sent_repeat_score': [0.0, 0.0, 0.0]}
{'number_ents': 0, 'ents': [], 'number_nouns': 6, 'nouns': ['author', 'finding', 'potential', 'development', 'vaccine', 'system'], 'number_sents': 1, 'avg_sent_len': 24.0, 'sent_repeat_score': [0.16666666666666663]}
{'number_ents': 0, 'ents': [], 'number_nouns': 6, 'nouns': ['conclusion', 'topic', 'effect', 'medication', 'treatment', 'authority'], 'number_sents': 2, 'avg_sent_len': 15.5, 'sent_repeat_score': [0.0, 0.0]}
{'number_ents': 0, 'ents': [], 'number_nouns': 7, 'nouns': ['introduction', 'study', 'efficacy', 'safety', 'drug', 'treatment', 'covid-'], 'number_sents': 1, 'avg_sent_len': 25.0, 'sent_repeat_score': [0.16000000000000003]}
{'number_ents': 0, 'ents': [], 'number_nouns': 6, 'nouns': ['covid-19', 'epicenter', 'uncertainty', 'health', 'health', 'problem'], 'number_sents': 3, 'avg_sent_len

In [44]:
print('Number named entities, number nouns, number of sentences, avg sentence length, repeat score')
for key in length_metric_1.keys():
  metric_load = length_metric_1[key]['metric']
  print(metric_load['number_ents'], metric_load['number_nouns'],
        metric_load['number_sents'],  metric_load['avg_sent_len'],
        metric_load['sent_repeat_score'])

Number named entities, number nouns, number of sentences, avg sentence length, repeat score
1 4 3 7.0 [0.0, 0.0, 0.0]
0 6 1 24.0 [0.16666666666666663]
0 6 2 15.5 [0.0, 0.0]
0 7 1 25.0 [0.16000000000000003]
0 6 3 8.666666666666666 [0.0, 0.2666666666666667, 0.0]
2 6 1 27.0 [0.18518518518518523]


In [45]:
for key in length_metric_2.keys():
  print(key)

The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will 
While everyone knows that a small number of people has cancer, the fact that doctors can treat any of them with a vaccine can help people recognize them by recognizing and responding to it. Some people also use certain treatments according to what may help. The Centers for Disease Control and Prevention (CDC) estimate that around 22 percent
The pandemic of pandemic has turned the whole globe across four continents. However, it is important to understand the impact of the pandemic by analysing the cumulative effects of the pandemic in the affected countries. It may not be possible to predict where countries have experienced and will increase the epidemic severity and deaths.
A range of remedies can help relieve severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) caused by acute respiratory syndrome coronavirus 2 (SARS-CoV-2). Here we report results of a trial of th

In [46]:
for key in length_metric_2.keys():
    print(key)
    print(len(key))

The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will 
114
While everyone knows that a small number of people has cancer, the fact that doctors can treat any of them with a vaccine can help people recognize them by recognizing and responding to it. Some people also use certain treatments according to what may help. The Centers for Disease Control and Prevention (CDC) estimate that around 22 percent
342
The pandemic of pandemic has turned the whole globe across four continents. However, it is important to understand the impact of the pandemic by analysing the cumulative effects of the pandemic in the affected countries. It may not be possible to predict where countries have experienced and will increase the epidemic severity and deaths.
339
A range of remedies can help relieve severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) caused by acute respiratory syndrome coronavirus 2 (SARS-CoV-2). Here we report results of a

In [47]:
print('Number named entities, number nouns, number of sentences, avg sentence length, repeat score')
for key in length_metric_2.keys():
  metric_load = length_metric_2[key]['metric']
  print(metric_load['number_ents'], metric_load['number_nouns'],
        metric_load['number_sents'],  metric_load['avg_sent_len'],
        metric_load['sent_repeat_score'])

Number named entities, number nouns, number of sentences, avg sentence length, repeat score
1 4 3 7.0 [0.0, 0.0, 0.0]
3 15 3 21.0 [0.19444444444444442, 0.0, 0.0]
1 12 3 19.333333333333332 [0.15384615384615385, 0.24, 0.050000000000000044]
0 13 2 24.0 [0.32258064516129037, 0.11764705882352944]
0 15 2 29.5 [0.1875, 0.09090909090909094]
1 14 3 17.0 [0.04166666666666663, 0.10526315789473684, 0.0]


In [48]:
for key in length_metric_3.keys():
  print(key)
  print(len(key))

The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will 
114
The coronavirus infection caused January 2019 (COVID-19) was first reported in Wuhan, China where a virus occurred at Wuhan, Hubei province in the central Sino-Chinese Province of Yunnan and mainland mainland China. In this work, we show that a single viral genome-wide replication (PFPC) variant containing the SARS group of coronaviruses can be found on different parts of the cell. We propose an algorithm called a pandemic-causing viral viral genome encoding a viral variant within that SARS-CoV-2 genome using the nucleotide-binding domain
544
The COVID-19 pandemic characterized by the epidemic of severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2), and the pandemic of coronavirus disease 2019 (COVID-19). Our study aims to investigate three aspects of severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) transmission among hospitalized COVID-19 patients 

In [49]:
for key in length_metric_3.keys():
  print(length_metric_3[key]['metric'])

{'number_ents': 1, 'ents': [this year], 'number_nouns': 4, 'nouns': ['flu', 'year', 'conference', 'vaccine'], 'number_sents': 3, 'avg_sent_len': 7.0, 'sent_repeat_score': [0.0, 0.0, 0.0]}
{'number_ents': 8, 'ents': [January 2019, Wuhan, China, Wuhan, Hubei, Sino-Chinese, Yunnan, China], 'number_nouns': 33, 'nouns': ['coronavirus', 'infection', 'January', 'COVID-19', 'Wuhan', 'China', 'virus', 'Wuhan', 'Hubei', 'province', 'Province', 'Yunnan', 'mainland', 'mainland', 'China', 'work', 'genome', 'replication', 'PFPC', 'variant', 'SARS', 'group', 'coronaviruse', 'part', 'cell', 'algorithm', 'genome', 'variant', 'SARS', 'CoV-2', 'genome', 'nucleotide', 'domain'], 'number_sents': 3, 'avg_sent_len': 33.666666666666664, 'sent_repeat_score': [0.15384615384615385, 0.05882352941176472, 0.2142857142857143]}
{'number_ents': 2, 'ents': [2019, three], 'number_nouns': 31, 'nouns': ['covid-19', 'pandemic', 'epidemic', 'syndrome', 'coronavirus', 'SARS', 'pandemic', 'coronavirus', 'disease', 'COVID-19',

In [50]:
print('Number named entities, number nouns, number of sentences, avg sentence length, repeat score')
for key in length_metric_3.keys():
  metric_load = length_metric_3[key]['metric']
  print(metric_load['number_ents'], metric_load['number_nouns'],
        metric_load['number_sents'],  metric_load['avg_sent_len'],
        metric_load['sent_repeat_score'])

Number named entities, number nouns, number of sentences, avg sentence length, repeat score
1 4 3 7.0 [0.0, 0.0, 0.0]
8 33 3 33.666666666666664 [0.15384615384615385, 0.05882352941176472, 0.2142857142857143]
2 31 4 22.75 [0.22580645161290325, 0.06666666666666665, 0.0, 0.0]
4 19 1 70.0 [0.2142857142857143]
1 18 5 18.4 [0.05555555555555558, 0.09523809523809523, 0.09999999999999998, 0.09523809523809523, 0.08333333333333337]
0 22 7 17.714285714285715 [0.05555555555555558, 0.04761904761904767, 0.0, 0.11538461538461542, 0.052631578947368474, 0.13043478260869568, 0.0]
